## Исследование надёжности заёмщиков

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.


### Шаг 1. Изучение датасета.

In [3]:
from pymystem3 import Mystem
import pandas as pd
from collections import Counter

m = Mystem()

In [2]:
credits = pd.read_csv("/datasets/data.csv")
credits.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
credits.shape

(21525, 12)

In [4]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
credits.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [6]:
credits["family_status"].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [7]:
credits["income_type"].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [8]:
credits["education"].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [9]:
credits["purpose"].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [10]:
credits.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [11]:
credits.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

### Вывод

1. Открыли файл, получили общую информацию о строках, столбцах и содержащихся в них типах данных.
2. Получили информацию, что в столбцах с общим доходом и трудовым стажем содержатся пропуски.
3. Просмотрели столбцы с категориальными данными и выяснили, что в столбце образования необходимо удалить дубликаты, а в столбце с целью кредита - категоризировать данные, поскольку категории повторяются, но описаны разными словами.

Т.е., в целом, ознакомились с данными

### Шаг 2. Предобработка данных

### Обработка пропусков

In [12]:
credits.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски содержатся в столбцах days_employed и income_type с количественными данными, причем их одинаковое количество. 
Рассмотрим внимательнее, от чего зависит появление NaNов

In [13]:
credits[credits["total_income"].isnull()].head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Судя по всему, пропущенные значения не зависят ни от каких других столбцов, т.к. встречаются с разными значениями в других столбца. Это случайные пропуски, значения которых зависят от другого столбца (days_employed или total_income). Т.к. для получения кредита информация о стаже и доходе является обязаетельной, то, вероятно, пропуски связаны не с тем, что люди не заполнили эти данные в анкете, а с какой-то технической ошибкой.

Поскольку столбец total_income нам нужен для дальнейшего анализа, заменим NaN на характерное значение. 
Предположим, что уровень дохода зависит от типа дохода (сотрудник, пенсионер, госслужащий и т.д.): сгруппируем данные дохода по типу дохода и найдем среднее арифметическое дохода для каждого из выделенных типов

In [14]:
income_type_grouped = credits.groupby("income_type")["total_income"].mean()
income_type_grouped

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

Заполним средним арифметическим, вычисленным для каждого типа дохода, NaNы в общем доходе

In [32]:
income_type_grouped.index

Index(['безработный', 'в декрете', 'госслужащий', 'компаньон', 'пенсионер',
       'предприниматель', 'сотрудник', 'студент'],
      dtype='object', name='income_type')

In [33]:
for income_type in income_type_grouped.index:
    credits.loc[credits["income_type"] == income_type, "total_income"] = credits.loc[credits["income_type"] == income_type, "total_income"].fillna(value = income_type_grouped.loc[income_type])
    

In [34]:
credits.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Отсутствующие значения в total_income заменены средними

Остались NaNы в колонке с трудовым стажем. Поскольку этот столбец нам не нужен для дальнейшего анализа, его можно удалить.

In [35]:
credits = credits.drop(columns = "days_employed")

In [36]:
credits.isna().sum()

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Итак, мы обнаружили отсутствующие значения в столбцах с трудовым стажем и общим доходом.
Пропуски считаем случайными.
В столбце с общим стажем заменили NaNы на среднее арифметическое значение, характерное для каждого типа дохода, а столбец с трудовым стажем удалили, т.к. он не нужен для анализа.

### Замена типа данных

In [37]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB


Данные в столбце с общим доходом имеют вещественный тип данных. Мы знаем методы to_numeric, который переводит строковые значения в вещественные, и метод astype, которым можно перевести вещественные числа в целочисленнй формат. Применим метод astype для столбца total_income

In [38]:
credits["total_income"] = credits["total_income"].astype("int64", errors = "raise")
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


Вывод

Применили метод astype для приведения вещественных чисел в столбце total_income к целочисленному формату.

### Обработка дубликатов

In [39]:
credits.columns

Index(['children', 'dob_years', 'education', 'education_id', 'family_status',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

In [41]:
credits["education"].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Из результата анализа видно, что используются одни и те же данные в разном регистре. Причина может быть в том, что в анкете наверняка не было указано, в каком регистре писать образование, поэтому люди писали, как считали нужным. 
Удалим дубликаты, приведя все данные в столбце с образованием к нижнему регистру с помощью метода lower

In [42]:
credits["education"] = credits["education"].str.lower()
credits["education"].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

### Вывод

Удалили дубликаты в столбце с образованием, применив метод lower

In [44]:
credits = credits.drop_duplicates().reset_index(drop = True)
credits.duplicated().sum()

0

Удалили грубые дубликаты

### Лемматизация

In [45]:
credits["purpose"].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Используем библиотеку pymystem3 для лемматизации значений в столбце с целями кредита. Для этого напишем функцию для лемматизации каждой строки, а результат добавим в новый столбец lemmatized_purpose

In [46]:
def credit_purpose(purpose):
    lemmatized_purpose = m.lemmatize(purpose)
    return lemmatized_purpose

credits["lemmatized_purpose"] = credits["purpose"].apply(credit_purpose)
credits

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatized_purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]"
21450,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]"
21451,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]"
21452,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]"


UPDATE по лемматизации ниже:

In [47]:
lemmas = ' '.join(credits["purpose"])
lemmatized_purpose = m.lemmatize(lemmas)

In [48]:
Counter(lemmatized_purpose).most_common(20)

[(' ', 55023),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904)]

### Вывод

### Категоризация данных

Для ответа на поставленные вопросы нам нужно провести категоризацию данных в столбцах с целями, наличием детей и уровнем дохода. 

Проанализироваов лемматизированные данные по столбку с целью кредита, можно выделить следующие категории: недвижимость, автомобиль, образование и свадьба.
Напишем функцию для того, чтобы выделить категории в столбце с лемматизированной целью кредита, и добавим категории в новый столбец purpose_groups.

In [49]:
def purpose_group(row):
    if "свадьба" in row:
        return "свадьба"
    elif "образование" in row:
        return "образование"
    elif "автомобиль" in row:
        return "автомобиль"
    elif "недвижимость" or "жилье":
        return "недвижимость"
credits["purpose_groups"] = credits["lemmatized_purpose"].apply(purpose_group)
credits["purpose_groups"].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_groups, dtype: int64

In [50]:
credits.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatized_purpose,purpose_groups
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",недвижимость
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость


Категоризируем заемщиков по наличию/отсутствию детей.

In [51]:
credits["children"].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [52]:
def children(row):
    if row == 0:
        return "нет"
    else:
        return "есть"
credits["children_categorized"] = credits["children"].apply(children)
credits.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatized_purpose,purpose_groups,children_categorized
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,есть
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,есть
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,нет
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,есть
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,нет
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",недвижимость,нет
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,нет
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование,нет
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,есть
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость,нет


In [53]:
credits["children_categorized"].value_counts()

нет     14091
есть     7363
Name: children_categorized, dtype: int64

Категоризируем заемщиков по уровню дохода. Разобьем общий доход на пять равных категорий.

Согласно статье https://www.rbc.ru/economics/18/06/2018/5b27bf829a794710796bbfe5, чтобы считаться средним классом в России (не считая Москвы, нужно зарабатывать не менее 60 тыс. в месяц)

In [56]:
x = 0.10
y = 0.99

credits['total_income'].quantile(q=[x,y])

0.10     78721.10
0.99    505511.63
Name: total_income, dtype: float64

Методом подбора по квантилям выяснили, что менее 10% людей имеют ежемесячный доход ниже квалифицируемого как средний класс, 89% принадлежат к среднему классу (ориентировочно), и только 0,01% процента богаче среднего класса

In [57]:
def income(i):
    if i < 80000:
        return "низкий доход"
    elif 80000 <= i <= 500000:
        return "средний класс"
    else:
        return "богатые"
credits["income_categorized"] = credits["total_income"].apply(income)
credits.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatized_purpose,purpose_groups,children_categorized,income_bins,income_categorized
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,есть,"(18422.063, 469654.4]",средний класс
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,есть,"(18422.063, 469654.4]",средний класс
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,нет,"(18422.063, 469654.4]",средний класс
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,есть,"(18422.063, 469654.4]",средний класс
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,нет,"(18422.063, 469654.4]",средний класс
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",недвижимость,нет,"(18422.063, 469654.4]",средний класс
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,нет,"(18422.063, 469654.4]",средний класс
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование,нет,"(18422.063, 469654.4]",средний класс
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,есть,"(18422.063, 469654.4]",средний класс
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость,нет,"(18422.063, 469654.4]",средний класс


### Вывод

Категоризировали следующие данные:
1. Цели кредита: недвижимость, образование, автомобиль, свадьба
2. Наличие/отсутствие детей
3. Уровень дохода разбили на три группы - низкий доход, средний класс и богатые.

### Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [59]:
credits.columns

Index(['children', 'dob_years', 'education', 'education_id', 'family_status',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose', 'lemmatized_purpose', 'purpose_groups',
       'children_categorized', 'income_bins', 'income_categorized'],
      dtype='object')

In [60]:
children_debt_rel = pd.pivot_tableincome_debt_rel = pd.pivot_table(index = "children_categorized", values = "debt", data = credits, aggfunc = ["mean", "count", "sum"])
children_debt_rel["debt_percentage"] = children_debt_rel[("mean", "debt")] *100
children_debt_rel

,mean,count,sum,debt_percentage
,debt,debt,debt,
children_categorized,,,,
есть,0.092082,7363,678,9.208203
нет,0.075438,14091,1063,7.543822


### Вывод

Не совсем очевидно, что можно считать зависимостью, но предположим, что для банка важен каждый процент, поэтому будем считать, что если разница составляет более 1%, то зависимость есть. С учетом этого предположения можно сделать вывод, что зависимость между наличием детей и возвратом кредита в срок существует: бездетные почти на 2% процента реже имеют задолженность по кредиту.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [61]:
credits.groupby("family_status").sum().sort_values("debt", ascending = False)

,children,dob_years,education_id,family_status_id,debt,total_income
family_status,,,,,,
женат / замужем,7890,537328,9934,0,931,2083796842
гражданский брак,2126,174630,3476,4151,388,693132160
Не женат / не замужем,806,107802,2269,11240,274,473676135
в разводе,547,54394,953,3585,85,202840330
вдовец / вдова,215,54187,898,1918,63,138631730


In [62]:
credits.groupby("family_status")["debt"].sum()

family_status
Не женат / не замужем    274
в разводе                 85
вдовец / вдова            63
гражданский брак         388
женат / замужем          931
Name: debt, dtype: int64

UPDATE по семейному положению ниже:

In [63]:
family_debt_rel = pd.pivot_table(index = "family_status", values = "debt", data = credits, aggfunc = ["mean", "count", "sum"])
family_debt_rel["debt_percentage"] = family_debt_rel[("mean", "debt")] *100
family_debt_rel.sort_values(by = ("mean", "debt"), ascending = False)

,mean,count,sum,debt_percentage
,debt,debt,debt,
family_status,,,,
Не женат / не замужем,0.097509,2810,274,9.750890
гражданский брак,0.093471,4151,388,9.347145
женат / замужем,0.075452,12339,931,7.545182
в разводе,0.071130,1195,85,7.112971
вдовец / вдова,0.065693,959,63,6.569343


### Вывод

В результате анализа можно сделать вывод, что зависимость между семейным положением и наличием задолженности по кредиту присутствует: самый большой процент невозврата кредита наблюдается у не женатых людей и людей, живущих в гражданском браке; у тех людей, которые женаты или в разводе, невозвратов по кредиту на 2% меньше, а у вдовцов - на 3%

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

UPDATE по уровню дохода ниже:

In [65]:
credits.columns

Index(['children', 'dob_years', 'education', 'education_id', 'family_status',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose', 'lemmatized_purpose', 'purpose_groups',
       'children_categorized', 'income_bins', 'income_categorized'],
      dtype='object')

In [66]:
income_debt_rel = pd.pivot_table(index = "income_categorized", values = "debt", data = credits, aggfunc = ["mean", "count", "sum"])
income_debt_rel["debt_percentage"] = income_debt_rel[("mean", "debt")] *100
income_debt_rel.sort_values(by = ("mean", "debt"), ascending = False)


,mean,count,sum,debt_percentage
,debt,debt,debt,
income_categorized,,,,
средний класс,0.081927,18956,1553,8.192657
низкий доход,0.076450,2276,174,7.644991
богатые,0.063063,222,14,6.306306


### Вывод

Из результатов анализа можно сделать вывод, зависимость между уровнем дохода и возвратом кредита в срок присутствует: люди из среднего класса (от 80,000 руб. в месяц до 500,000 руб. в месяц) почти на 2% процента чаще имеют просрочку по кредиту, чем люди богатые (зп - более 500,000 рублей в месяц), а люди с низким доходом (до 80,000 рублей в месяц) имеют просрочку на 1% чаще, чем богатые.

- Как разные цели кредита влияют на его возврат в срок?

In [67]:
credits.groupby("purpose_groups")["debt"].sum()

purpose_groups
автомобиль      403
недвижимость    782
образование     370
свадьба         186
Name: debt, dtype: int64

In [68]:
credits.columns

Index(['children', 'dob_years', 'education', 'education_id', 'family_status',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose', 'lemmatized_purpose', 'purpose_groups',
       'children_categorized', 'income_bins', 'income_categorized'],
      dtype='object')

In [69]:
purpose_debt_rel = pd.pivot_table(index = "purpose_groups", values = "debt", data = credits, aggfunc = ["mean", "count", "sum"])
purpose_debt_rel["debt_percentage"] = purpose_debt_rel[("mean", "debt")] *100
purpose_debt_rel.sort_values(by = ("mean", "debt"), ascending = False)

,mean,count,sum,debt_percentage
,debt,debt,debt,
purpose_groups,,,,
автомобиль,0.093590,4306,403,9.359034
образование,0.092200,4013,370,9.220035
свадьба,0.080034,2324,186,8.003442
недвижимость,0.072334,10811,782,7.233373


In [70]:
purpose_debt_rel = pd.pivot_table(index = "purpose_groups", values = "debt", columns = "children_categorized", data = credits, aggfunc = "mean")
purpose_debt_rel

children_categorized,есть,нет
purpose_groups,,
автомобиль,0.109514,0.085413
недвижимость,0.081884,0.067289
образование,0.102845,0.086677
свадьба,0.089421,0.075163


In [71]:
purpose_debt_rel = pd.pivot_table(index = "purpose_groups", values = "debt", columns = "income_categorized", data = credits, aggfunc = "mean")
purpose_debt_rel

income_categorized,богатые,низкий доход,средний класс
purpose_groups,,,
автомобиль,0.075000,0.082960,0.095026
недвижимость,0.047244,0.074913,0.072357
образование,0.117647,0.070455,0.094660
свадьба,0.047619,0.082645,0.080058


### Вывод

В результате анализа делаем вывод, что чаще всего задолженность по кредиту имеют люди, приобретающие автомобили в кредит (9%), а также люди с целью получения образования (также 9%). На 1% реже просрочка бывает, если цель получения кредита - свадьба, и на 2% реже - если целью является недвижимость. 

Если учитывать одновременно цель кредита и наличие детей, то можно сделать вывод, что при любой цели кредита бездетные в среднем на 2% реже задерживают выплаты по кредиту, что соответствует выводу, сделанному по первому вопросу.

Если рассматривать одновременно цель кредита и уровень дохода, то получаем следующее: если цель - недвижимость или свадьба, то богатые имеют просрочку по кредиту на 3% реже, чем средний класс и люди с низким доходом; если цель кредита - образование, то богатые имеют задолженность на 4% чаще, чем люди с низким доходом, и на 2% чаще, чем люди из среднего класса.

### Шаг 4. Общий вывод

Изучив файл, мы осуществили предобработку данных, а затем ответили на поставленные вопросы.
Предобработка данных включала:
- поиск пропусков, заполнение пропусков в столбце доход средним арифметическим, посчитанным по каждому типу дохода, удаление пропусков в столбце с трудовым стажем, поскольку он нам не нужен для анализа
- заменили вещественный тип данных на целочисленный в столбце с общим доходом
- удалили дубликаты в столбце с образованием, применив метод str.lower()
- лемматизировали данные в столбце с целями кредита
- категоризировали данные в столбцах с детьми, общим доходом и целью кредита, чтобы ответить на поставленные вопросы
Анализ полученных данных дал возможность сделать следующие выводы:
1. Связь между наличием детей и возвратом кредита в срок есть: бездетные на почти 2% процента реже имеют задолженность по кредиту;
2. Связь между семейным положением и возвратом кредита в срок есть: самый большой процент невозврата кредита наблюдается у не женатых людей и людей, живущих в гражданском браке; у тех людей, которые женаты или в разводе, невозвратов по кредиту на 2% меньше, а у вдовцов - на 3%;
3. Зависимость между уровнем дохода и возвратом кредита в срок есть: люди из среднего класса (от 80,000 руб. в месяц до 500,000 руб. в месяц) почти на 2% процента чаще имеют просрочку по кредиту, чем люди богатые (зп - более 500,000 рублей в месяц), а люди с низким доходом (до 80,000 рублей в месяц) имеют просрочку на 1% чаще, чем богатые;
4. Цели кредита бывают - недвижимость, образование, автомобиль и свадьба. чаще всего задолженность по кредиту имеют люди, приобретающие автомобили в кредит (9%), а также люди с целью получения образования (также 9%). На 1% реже просрочка бывает, если цель получения кредита - свадьба, и на 2% реже - если целью является недвижимость.